# The Devil and the Coin Flip Game

>You're playing a game with the devil, with your soul at stake. You're sitting at a circular table which has 4 coins, arranged in a diamond, at the 12, 3, 6, and 9 o'clock positions. We'll number those as positions 0, 1, 2, and 3, respectively. You are blindfolded, and can never see the coins or the table.

>Your goal is to get all 4 coins showing heads, by telling the devil the position number(s) of some coins to flip. We call this a "move" on your part. The devil must faithfully perform the requested flips, but may first sneakily rotate the table any number of quarter-turns, so that the coins are in different positions. The game ends when 4 heads come up and you win; until then you  keep making moves.

> Example: You tell the devil to flip positions 0 and 2 (the 12 o'clock and 6 o'clock positions). The devil could leave the table unrotated (or could rotate it a half-turn), and then flip the two coins that you specified. Or the devil could rotate the table a quarter turn in either direction, and then flip the coins that are now in the 12 o'clock and 6 o'clock locations, which are the two other coins from the ones you specified.  You won't know which of these actions the devil took.

> What is a shortest sequence of moves that is guaranteed to win, no matter what the initial state of the coins, and no matter what rotations the devil applies?

# Analysis

The player, being blindfolded, does not know the true state of the coins. So the player should represent what is known: the *set of possible states* of the coins. We call this a *belief state*. At the start of the game, each of the four coins could be either heads or tails, so that's 2<sup>4</sup> = 16 possibilities in the belief state. 

Each possible move, such as `{0, 2}`, which means "flip the 12 o'clock and 6 o'clock positions," updates the belief state as follows: for every coin sequence in the current belief state, rotate it in every possible way, and then flip the appropriate coins. Collect all these results together to form the new belief state. To solve the game, I need to come up with a sequence of moves that ends in a belief state consisting of just `{'HHHH'}`. I want it to be a shortest possible sequence, so a breadth-first search seems right. The search space will be tiny, so compute time will be trivial; the only issue is specifying the domain correctly. (To increase the chance of getting it correct, I won't try to do anything fancy, such as noticing that some sequences are rotational variants of other sequences.)


# Implementation Choices

Here are the main concepts, and my implementation choices:

- `Coins`: A *coin sequence* is represented as a `str` of four characters, such as `'HTTT'`. 
- `Belief`: A *belief state* is represented as a `frozenset` of `Coins` (frozen so that it can be hashed in a `set`).
- `initial_belief`: The set of all possible coin sequences.
- `rotations`: The function `rotations(coins)` returns the set of all 4 rotations of the coin sequence.
- `update`: The function `update(belief, move)` retuns an updated belief state, representing all the possible coin sequences that could result from any devil rotation followed by the specified flip(s). (But don't flip `'HHHH'`, because the game would have already ended.)
- `flip`: The function `flip(coins, move)` flips the specified positions within the coin sequence.

In [1]:
from collections import deque, Counter
from itertools   import product, combinations
import random

Coins = ''.join # Function to make a 4-element Coin Sequence, such as 'HHHT'

Belief = frozenset

initial_belief = Belief(map(Coins, product('HT', repeat=4)))

def rotations(coins): return {coins[r:] + coins[:r] for r in range(4)}

def update(belief, move):
    "Update belief: consider all possible rotations, then flip."
    return Belief((flip(c, move) if c != 'HHHH' else c)
                  for coins in belief
                  for c in rotations(coins))

def flip(coins, move):
    "Flip the coins in the positions specified by the move."
    coins = list(coins) # Need a mutable sequence
    for i in move:
        coins[i] = ('H' if coins[i] == 'T' else 'T')
    return Coins(coins)

Let's try out some of the functions to see if they look right:

In [2]:
flip('HHHT', {0, 2})

'THTT'

In [3]:
rotations('HHHT')

{'HHHT', 'HHTH', 'HTHH', 'THHH'}

In [4]:
initial_belief

frozenset({'HHHH',
           'HHHT',
           'HHTH',
           'HHTT',
           'HTHH',
           'HTHT',
           'HTTH',
           'HTTT',
           'THHH',
           'THHT',
           'THTH',
           'THTT',
           'TTHH',
           'TTHT',
           'TTTH',
           'TTTT'})

In [5]:
update(initial_belief, {0, 1, 2, 3})

frozenset({'HHHH',
           'HHHT',
           'HHTH',
           'HHTT',
           'HTHH',
           'HTHT',
           'HTTH',
           'HTTT',
           'THHH',
           'THHT',
           'THTH',
           'THTT',
           'TTHH',
           'TTHT',
           'TTTH'})

That says that if we flip all 4 coins, we eliminate the possibility of 4 tails, cutting the possibilities down from 16 to 15.

Everything looks good so far. One more thing: we need to find all subsets of the 4 positions:

In [6]:
def powerset(sequence): 
    "All subsets of a sequence."
    return [set(c) 
            for r in range(len(sequence) + 1)
            for c in combinations(sequence, r)]

In [7]:
powerset(range(4))

[set(),
 {0},
 {1},
 {2},
 {3},
 {0, 1},
 {0, 2},
 {0, 3},
 {1, 2},
 {1, 3},
 {2, 3},
 {0, 1, 2},
 {0, 1, 3},
 {0, 2, 3},
 {1, 2, 3},
 {0, 1, 2, 3}]

# Search for a Solution

The function `search` does a breadth-first search starting
at the initial `belief` state and applying a sequences of moves, trying to
find a path that leads to the goal belief state `{'HHHH'}` (meaning that the only possibility is 4 heads).
As is typical for search algorithms, we build a search tree, keeping a queue of tree `nodes` to consider, where each 
node consists of a path (a sequence of moves) and a resulting belief state. We also keep track, in `explored`, of
the states we have already explored, so that we don't have to revisit them.
 

In [8]:
def search(start=initial_belief):
    "Breadth-first search from starting belief state using moves."
    explored = set()
    q = deque([Node([], start)])
    while q:
        (path, belief) = q.popleft()
        if belief == {'HHHH'}:
            return path
        for move in powerset(range(4)):
            belief2 = update(belief, move)
            if belief2 not in explored:
                explored.add(belief2)
                q.append(Node(path + [move], belief2))
                
def Node(path, belief): return (path, belief)

In [9]:
search()

[{0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3},
 {0, 1},
 {0, 1, 2, 3},
 {0, 2},
 {0, 1, 2, 3}]

That's a 15-move sequence that is guaranteed to lead to a win. 

# Verifying the Solution

Do I believe the solution is correct?  Playing with paper and pencil, it does appear to work. A colleague did the puzzle and got the same answer. And here's further validation: The function `random_devil` takes a sequence of moves and plays those moves with a devil that chooses randomly, returning the number of moves it takes until the player wins:

In [10]:
def random_devil(moves):
    "A random devil responds to moves; return the number of moves until win, or None."
    coins = random.choice(list(initial_belief))
    if coins == 'HHHH':
        return 0
    for (i, move) in enumerate(moves, 1):
        coins = flip(random.choice(list(rotations(coins))), move)
        if coins == 'HHHH': 
            return i

There are 16 coin sequences so let's call `random_play` 16,000 times, and count the results:

In [11]:
moves = search()

Counter(random_devil(moves) for _ in range(16000))

Counter({0: 1037,
         1: 1006,
         2: 1011,
         3: 998,
         4: 980,
         5: 976,
         6: 1057,
         7: 1043,
         8: 963,
         9: 1021,
         10: 966,
         11: 1012,
         12: 995,
         13: 975,
         14: 990,
         15: 970})

This says that the player won all 16,000 times. (If the player ever lost, there would have been an entry for `None` in the Counter.)
The remarkable thing, which I can't explain, is that there are very nearly 1,000 results for each of the counts from 0 to 15. Can you explain that?

# Visualizing the Solution

How does the solution work? One answer is "it takes care of all possibilities." But it would be nice to gain more insight. I'll print a table showing the belief state after each move. But I will do two things to make the belief state easier to read. First, I will *canonicalize* the belief state to be independent of rotations. By that I mean that if the belief stae contains `{'HHHT', 'HHTH', 'HTHH', 'THHH'}`, which are all rotations of each other,  I will only print one of them. (Arbitrarily, I choose the one that comes first alphabetically, `'HHHT'`.) This gets us down from 16 coin sequences in the belief state to six.
Second, I will print each coin sequence in its own column, so that they line up and it is easy to see when a particular coin sequence disappears from the belief state

In [12]:
def show(moves):
    "For each move, print the move number, move, and belief state."
    belief = initial_belief
    show_line(0, 'start', belief)
    for (i, move) in enumerate(moves, 1):
        belief = update(belief, move)
        show_line(i, move, belief)
        
def join(items, sep=''): return sep.join(map(str, items))
          
def canonical(belief): return sorted(set(min(rotations(coins)) for coins in belief))

def show_line(i, move, belief, order=canonical(initial_belief)):
    "Print the move number, move, and belief state."
    ordered_belief = [(coins if coins in belief else '    ')
                      for coins in order]
    print('{:2} {:5} {}'.format(i, join(move), join(ordered_belief, ' ')))

In [13]:
show(search())

 0 start HHHH HHHT HHTT HTHT HTTT TTTT
 1 0123  HHHH HHHT HHTT HTHT HTTT     
 2 02    HHHH HHHT HHTT      HTTT TTTT
 3 0123  HHHH HHHT HHTT      HTTT     
 4 01    HHHH HHHT      HTHT HTTT TTTT
 5 0123  HHHH HHHT      HTHT HTTT     
 6 02    HHHH HHHT           HTTT TTTT
 7 0123  HHHH HHHT           HTTT     
 8 0     HHHH      HHTT HTHT      TTTT
 9 0123  HHHH      HHTT HTHT          
10 02    HHHH      HHTT           TTTT
11 0123  HHHH      HHTT               
12 01    HHHH           HTHT      TTTT
13 0123  HHHH           HTHT          
14 02    HHHH                     TTTT
15 0123  HHHH                         


We can see that every odd-numbered move flips all four coins to eliminate the possibility of `TTTT`.
We can also see that moves 2, 4, and 6 flip two coins and have the effect of eventually eliminating the two "two heads" sequences from the belief state, and then move 8 flips a single coin, thus eliminating the "three heads" and "one heads" sequences, and bringing back the "two heads" possibilities. Repeating moves 2, 4, and 6 in moves 10, 12, and 14 then re-eliminates the "two heads", and move 15 gets rid of the last possibility.